# Hyperparameter search across optimizer, learning rate, weight decay

In [1]:
# Global imports
import os
import json
import sys
sys.path.insert(0, 'src')

In [2]:
# Local imports
from utils.util import read_json, informal_log
from train import main as train_fn
from test import main as test_fn
from parse_config import ConfigParser
from data_loader import data_loaders

In [11]:
# architectures = [
#     "vgg11_bn",
#     "vgg13_bn",
#     "vgg16_bn",
#     "vgg19_bn",
#     "resnet18",
#     "resnet34",
#     "resnet50",
#     "densenet121",
#     "densenet161",
#     "densenet169",
#     "mobilenet_v2",
#     "googlenet",
#     "inception_v3"]

# optimizers = ["Adam", "SGD"]
# learning_rates = [0.1, 0.01, 1e-3, 1e-4, 1e-5, 1e-6]
# weight_decays = [0, 0.1, 0.01, 1e-3, 1e-4]
# n_epochs = 30

config_path = 'configs/cinic10_imagenet_train_hyperparameter_search.json'
config_dict = read_json(config_path)



# Test values

architectures = ["vgg11_bn"]
optimizers = ["Adam", "SGD"]
learning_rates = [0.1, 0.01]
weight_decays = [0, 0.1]
n_epochs = 15

/n/fs/ac-editing/model-editing


In [4]:
# Initialize data loaders
data_loader_args = dict(config_dict["data_loader"]["args"])
train_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    shuffle=True,
    split='train')
val_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    shuffle=False,
    split='valid')
test_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    shuffle=False,
    split='test')

In [ ]:
for architecture in architectures:
    config_dict.update({"name": "HyperParameterSearch_{}".format(architecture)})
    summary_file_path = os.path.join(
        config_dict["trainer"]["save_dir"], 
        config_dict["name"], 
        'summary.txt')

    informal_log("Train data path: {}".format(train_data_loader.get_data_dir()))
    informal_log("Validation data path: {}".format(val_data_loader.get_data_dir()))
    informal_log("Test data path: {}".format(test_data_loader.get_data_dir()))

    informal_log("Test results for {}...".format(architecture), summary_file_path)
    
    # Set number of epochs
    config_dict["trainer"].update({"epochs": n_epochs})
    # Update model architecture information

    for key, value in config_dict["arch"]["args"].items():
        config_dict["arch"]["args"].update({key: value.format(architecture)})

    print(config_dict["name"])
    print(config_dict["arch"])
    for optimizer in optimizers:
        config_dict["optimizer"].update({"type": optimizer})
        for learning_rate in learning_rates:
            config_dict["optimizer"]["args"].update({"lr": learning_rate})
            for weight_decay in weight_decays:
                
                
                config_dict["optimizer"]["args"].update({"weight_decay": weight_decay})
                print(config_dict["optimizer"])
                run_id = "optim_{}-lr_{}-wd_{}".format(
                    config_dict["optimizer"]["type"],
                    config_dict["optimizer"]["args"]["lr"],
                    config_dict["optimizer"]["args"]["weight_decay"]
                )
                
                # Log parameters in summary
                informal_log("Optimizer: {} \tLearning Rate: {} \tWeight Decay: {}".format(
                    optimizer, learning_rate, weight_decay
                ), summary_file_path)
                informal_log("N_epochs: {}".format(n_epochs), summary_file_path)
                # Create config object and train
                config = ConfigParser(config_dict, run_id=run_id)
                train_fn(config,
                    train_data_loader=train_data_loader,
                    val_data_loader=val_data_loader)
                
                # Obtain best model and run on test set
                best_ckpt_path = os.path.join(config.save_dir, 'model_best.pth')
                informal_log("Best checkpoint at {}".format(best_ckpt_path))
                test_run_id = os.path.join(run_id, 'test')
                config_test = ConfigParser(config_dict, run_id=test_run_id, resume=best_ckpt_path)
                test_results = test_fn(config_test, test_data_loader=test_data_loader)
                
                informal_log("Test results:\n{}".format(test_results), summary_file_path)
                informal_log("\n---***---\n", summary_file_path)
'''
for each arch:
    change config name=HyperParamSearch_{arch.type}, arch.type, arch.checkpoint_path,
    for each optimizer:
        set config optimizer.type
        for each lr:
            set config optimizer.args.lr
            for each weight decay:
                setconfig.optimizer.weight_decay
                config.run_id = {dataset}_{arch.type}_{optim}_{lr}_{weight_decay}
                config = ConfigParser(config)
                train.main(config)
                

'''


Train data path: data/cinic-10-imagenet/train
Validation data path: data/cinic-10-imagenet/valid
Test data path: data/cinic-10-imagenet/test
Test results for vgg11_bn...
HyperParameterSearch_vgg11_bn
OrderedDict([('type', 'CIFAR10PretrainedModel'), ('args', OrderedDict([('type', 'vgg11_bn'), ('checkpoint_path', 'external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg11_bn.pt')]))])
OrderedDict([('type', 'Adam'), ('args', OrderedDict([('lr', 0.1), ('weight_decay', 0), ('amsgrad', False)]))])
Optimizer: Adam 	Learning Rate: 0.1 	Weight Decay: 0
N_epochs: 15
Created CIFAR10PretrainedModel model with 28149514 trainable parameters
Restored weights from external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg11_bn.pt
Checkpoint save directory: saved/HyperParameterSearch_vgg11_bn/optim_Adam-lr_0.1-wd_0/models
Train Epoch: 1 [0/70000 (0%)] Loss: 1.805037
Train Epoch: 1 [4096/70000 (6%)] Loss: 2.379120
Train Epoch: 1 [8192/70000 (12%)] Loss: 2.328338
Train Epoch: 1 [12288/70000 (18%)] 

In [9]:
# print stuff
print(config["optimizer"])
best_ckpt_path = os.path.join(config.save_dir, 'model_best.pth')
print(best_ckpt_path)

OrderedDict([('type', 'Adam'), ('args', OrderedDict([('lr', 0.1), ('weight_decay', 0), ('amsgrad', False)]))])
saved/HyperParameterSearch_vgg11_bn/optim_Adam-lr_0.1-wd_0/models/model_best.pth
